In [8]:
from selenium import webdriver
from bs4 import BeautifulSoup
from collections import defaultdict
import time
import datetime
import os
import json
from krwordrank.word import KRWordRank
from wordcloud import WordCloud

import pandas as pd

In [9]:
import re

FIND_NUM = 10                                   # 탐색하고자 하는 횟수, 개당 20게 게시물
YOUR_ID =                          # 에브리타임 아이디
YOUR_PASSWORD =                       # 에브리타임 비밀번호
BOARD_URL = 'https://everytime.kr/370439/p/'   # 에브리타임 해당 학교 자유게시판 주소
def tree(): 
    return defaultdict(tree)

driver = webdriver.Chrome('./chromedriver.exe')
driver.implicitly_wait(10)
driver.get('https://everytime.kr/login')

# 접속
driver.find_element_by_xpath('//*[@id="container"]/form/p[1]/input').send_keys(YOUR_ID)   # 아이디
driver.find_element_by_xpath('//*[@id="container"]/form/p[2]/input').send_keys(YOUR_PASSWORD)   # 비밀번호
driver.find_element_by_xpath('//*[@id="container"]/form/p[3]/input').click()                # 로그인 버튼
time.sleep(2)
driver.find_element_by_xpath('//*[@id="container"]/div[4]/div[1]/div/h3/a').click()          # 자유게시판 클릭

everytime_link = list() # 링크 리스트
fail_link = list() # 실패 리스트
page_number = 2
content_number = 0


if not os.path.isdir("./Result/"):
    os.mkdir("./Result/") 
    
for i in range(FIND_NUM):
    time.sleep(7)    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    content = soup.findAll('article')

    for url in content:
        find_url = url.find('a', attrs={'class', 'article'}).get('href')
        everytime_link.append(find_url)
    time.sleep(2)
    driver.get(BOARD_URL + str(page_number)) # 자유게시판
    page_number = page_number + 1

with open('./everytime_link.txt', 'w') as fileobject:
    for join_link in everytime_link:
        fileobject.write(join_link)
        fileobject.write('\n')



for url in everytime_link:
        
    time_now = datetime.datetime.now()      #현재 시간 저장
    json_data = dict()
    comment_text = list()
    comment_time = list()
    json_data['comment_text'] = list()
    json_data['comment_time'] = list()
    
    try:
        driver.get('https://everytime.kr' + url)
        time.sleep(5)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        # append 로 수정할것
        title = soup.find('h2', attrs={'class', 'large'}).get_text() 
        title = re.sub('[^가-힣0-9a-zA-Z\\s]','',title)
        text = soup.find('p', attrs={'class', 'large'}).get_text()
        text = re.sub('[^가-힣0-9a-zA-Z\\s]','',text)
        text_time = soup.find('time', attrs={'class', 'large'}).get_text()
        
    #    try:
  #          comment = soup.findAll('article')
   #         comment = re.sub('[^가-힣0-9a-zA-Z\\s]','',comment)
#
    #        for content in comment:
    #            comment_text.append(content.find('p').get_text())
   #             comment_time.append(content.find('time').get_text())
   #     except:
   #         pass                   #댓글없음
        
        
        json_data['title'] = title
        json_data['text'] = text
        json_data['tex_time'] = text_time
        #json_data['now_time'] = str(time_now)
        
        
   #     json_data['comment_text'] = comment_text
  #      json_data['comment_time'] = comment_time
        
    except Exception as e:
        print(e)
        fail_link.append(url)
        continue

    with open('./Result/' + 'text' + str(content_number) + '.txt', 'w') as fileobject:
        json.dump(json_data, fileobject)
        content_number = content_number + 1
            
            
with open('./fail_url.txt', 'w') as fileobject:
    for join_link in fail_link:
        fileobject.write(join_link)
        fileobject.write('\n')
            
driver.close()

'NoneType' object has no attribute 'get_text'


In [10]:
# https://wikidocs.net/39 참조

def search(dirname):        # 디렉토리 속 파일 경로 리스트 가져오기 ex : ./Result/file.txt
    file_name_list = list()
    filenames = os.listdir(dirname)
    for filename in filenames:
        full_filename = os.path.join(dirname, filename)
        file_name_list.append(full_filename)
    return file_name_list

def file_read(file_name_list):  # json 으로 해당 경로 파일 가져오기
    data = list()
    for file_path in file_name_list:
        with open(file_path, 'r') as file_point:
            data.append(json.load(file_point))
    return data

In [11]:
file_list = search('./Result/')
data = file_read(file_list)

#print(file_list)
print(data[0].keys())
print(data[4]['title'])
print(data[0]['text'])

dict_keys(['comment_text', 'comment_time', 'title', 'text', 'tex_time'])
남자분들 조언좀



In [12]:
results=pd.DataFrame(data)
results.comlumns=['title', 'text', 'tex_time', 'now_time']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [13]:
results.to_csv('everytime1.csv',encoding='cp949')

In [ ]:
# https://lovit.github.io/nlp/2018/04/17/word_cloud/ 참조

wordrank_extractor = KRWordRank(
    min_count = 10, # 단어의 최소 출현 빈도수 (그래프 생성 시)
    max_length = 15, # 단어의 최대 길이
    verbose = True
    )

beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10

input_text = str()
text = list()
for content in data:
    text.append(content['text'])
    text.append(content['title'])
    
    for comment in content['comment_text']:
        text.append(comment)

input_text = ' '.join(text)

In [ ]:
keywords = None
rank = ''
graph = None
keywords, rank, graph = wordrank_extractor.extract(text, beta, max_iter)

In [ ]:
for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:60]:
    print('%8s:\t%.4f' % (word, r))